In [2]:
import requests
import time
import pandas as pd

API_KEY = "73af62f1-882f-4c88-a477-ce6f38adde01"

def make_api_request(url):
    headers = {"api_key": API_KEY}
    
    while True:
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            return response.json()
        except requests.HTTPError as e:
            if e.response.status_code == 429:
                retry_after = int(e.response.headers.get('Retry-After', 60))
                print(f"Rate limit reached. Waiting for {retry_after} seconds before retrying...")
                time.sleep(retry_after)
            else:
                print(f"HTTP error occurred: {e}")
                return None
        except requests.RequestException as e:
            print(f"An error occurred: {e}")
            return None

def get_supported_exchanges():
    url = "https://api.coinalyze.net/v1/exchanges"
    return make_api_request(url)

def create_exchanges_dataframe(exchanges):
    return pd.DataFrame(exchanges)

def get_supported_future_markets():
    url = "https://api.coinalyze.net/v1/future-markets"
    return make_api_request(url)

def create_supported_futures_markets_dataframe(future_markets):
    df = pd.DataFrame(future_markets)
    
    # Convert expire_at from milliseconds to datetime
    df['expire_at'] = pd.to_datetime(df['expire_at'], unit='ms')
    
    return df

def filter_btc_perpetual_futures(futures_df):
    # Filter the DataFrame for BTC base asset and perpetual futures
    btc_perpetual_df = futures_df[(futures_df['base_asset'] == 'BTC') & (futures_df['is_perpetual'] == True)]
    
    return btc_perpetual_df

def main():
    # Fetch and process exchanges
    exchanges = get_supported_exchanges()
    if exchanges:
        exchanges_df = create_exchanges_dataframe(exchanges)
        print("Supported Exchanges:")
        print(exchanges_df)
        exchanges_df.to_csv('supported_exchanges.csv', index=False)
        print("Exchanges DataFrame saved to 'supported_exchanges.csv'")
    else:
        print("Failed to retrieve supported exchanges.")

    # Fetch and process future markets
    future_markets = get_supported_future_markets()
    if future_markets:
        futures_df = create_supported_futures_markets_dataframe(future_markets)
        print("\nSupported Future Markets:")
        print(futures_df)
        futures_df.to_csv('supported_future_markets.csv', index=False)
        print("Future Markets DataFrame saved to 'supported_future_markets.csv'")
        
        # Filter for BTC perpetual futures
        btc_perpetual_df = filter_btc_perpetual_futures(futures_df)
        print("\nBTC Perpetual Futures:")
        print(btc_perpetual_df)
        btc_perpetual_df.to_csv('btc_perpetual_futures.csv', index=False)
        print("BTC Perpetual Futures DataFrame saved to 'btc_perpetual_futures.csv'")
    else:
        print("Failed to retrieve supported future markets.")

if __name__ == "__main__":
    main()

Supported Exchanges:
                   name code
0              Poloniex    P
1                Vertex    V
2              Bitforex    D
3                Kraken    K
4               Bithumb    U
5              Bitstamp    B
6              BitFlyer    L
7            BtcMarkets    M
8                 Bit2c    I
9        MercadoBitcoin    E
10  Independent Reserve    N
11               Gemini    G
12              Gate.io    Y
13              Deribit    2
14                  OKX    3
15             Coinbase    C
16             Bitfinex    F
17                 Luno    J
18               BitMEX    0
19               Phemex    7
20                WOO X    W
21                Huobi    4
22                 dYdX    8
23                Bybit    6
24              Binance    A
Exchanges DataFrame saved to 'supported_exchanges.csv'

Supported Future Markets:
                 symbol exchange symbol_on_exchange base_asset quote_asset  \
0             MNTUSDT.6        6            MNTUSDT        MNT   